# Lazy Theta Star Online

### Contents

1. **Introduction**
    1. [Why the problem is important](#intro)
    1. [What is lacking in the current knowledge / State of the art](#stateOfTheArt)
    1. [Objectives of the study & research question ](#researchQuestion)
    1. [Contribution](#contribution)

1. **Methods**
    1. [Installation](#installation)
    1. [Run](#run)
    1. [Study design](#studyDesign)
    1. [Study variables](#studyVariables)

<a id='intro'></a>
#### 1.A Why the problem is important

<a id='stateOfTheArt'></a>
#### 1.B What is lacking in the current knowledge / State of the art

Not comparing with MoveIt! because it is built for robotic arms and it would be a study in it's own right to find out the proper parameters. A nice playlist of intro videos exists. https://www.youtube.com/playlist?list=PLz8Xi5I9bhzKuFfcQxWvMm9sVCR5Wng90 

Another option is to use the underlying OMPL. But the obstacle avoidance is in MoveIt! .

Metrics from Helen and Lactombe 2002.

<a id='researchQuestion'></a>
#### 1.C Objectives of the study & research question

**Research question** What modifications need to be done to use Lazy theta * online and in the real world.



<a id='contribution'></a>
#### 1.D Contribution

- Online
- Receding horizon?
- Real world adaptation:
    - Free space corridor
    - Start & goal in same voxel
    - Unknown space as obstacle?
    - Scale?
    - Security margin & neighbor generation
- In case of Information Gain
    - Show with the sensor specs that the error os measurements is uniform throuhout the langer range.
    - [Disc area](https://www.mathopenref.com/arcsector.html)
    - [Is point part of arc](https://stackoverflow.com/questions/6270785/how-to-determine-whether-a-point-x-y-is-contained-within-an-arc-section-of-a-c)
    

**Security margin & neighbor generation** 

Using security margin to generate neighbors instead of octree resolution. [for calculations look in spreadsheet](https://docs.google.com/spreadsheets/d/1c4LzxBBLpTx6qQF26tpFr8tkmEM5zIi43MLb72PWlvE/edit#gid=0) 

When there is a large mismatch between security margin and the resolution there is no point generating neighbors for maximum resolution steps when most of them will be discarded. 

Instead a variable fraction of the security margin (this can be explored - know what is the amount of paths that cannot be found with this that can be found with resolution) (having as minimum lenght the octree resolution) will be sampled for neighbors. 

This reduces the number of neighbors while scaling with the size of the voxel. 
It is not possible to use this method of neighbor generation for both UpdateVertex and in the main loop because during the implementation of the algorithm the assumption that every maximum.

(Sparse version) Using the actual size of the neighbors did not in fact reduce computation time because if bookkeeping. 
    


<a id='installation'></a>
#### 2.A Installation

There are 3 environments with px4: sitl, hitl and real world.
The instructions for installation are :
- stil  [Gazebo simulation - Try 9](Paper%20-%20Exploration%20real%20world.ipynb#try9)
- hitl [Hardware in the loop - Try 2 just upboard and laptop (no docker)](Paper%20-%20Exploration%20real%20world.ipynb#hitl)
    1. In laptop Firmware
        1. Install mavros-extras
        1. `cd ~`
        1. `wget https://raw.githubusercontent.com/PX4/Devguide/master/build_scripts/ubuntu_sim_ros_gazebo.sh`
        1. `chmod +x ubuntu_sim_ros_gazebo.sh`
        2. `source ubuntu_sim_ros_gazebo.sh`
        1. `git checkout v1.7.0-rc4`
        1. `make posix_sitl_default gazebo`
        1. `source ~/Flying_Octomap_code/devel/setup.bash`
        1. `source Tools/setup_gazebo.bash ~/src/Firmware ~/src/Firmware/build/posix_sitl_default`
        1. `export ROS_PACKAGE_PATH=$ROS_PACKAGE_PATH:~/src/Firmware`
        1. `export ROS_PACKAGE_PATH=$ROS_PACKAGE_PATH:~/src/Firmware/Tools/sitl_gazebo`
        1. `cp ~/Flying_Octomap_code/src/architecture/Tools/iris.xacro ~/src/Firmware/Tools/sitl_gazebo/models/rotors_description/urdf/`
        1. `cp ~/Flying_Octomap_code/src/architecture/Tools/iris_base.xacro ~/src/Firmware/Tools/sitl_gazebo/models/rotors_description/urdf/`
        1. `python ~/src/Firmware/Tools/sitl_gazebo/scripts/xacro.py -o ~/src/Firmware/Tools/sitl_gazebo/models/rotors_description/urdf/iris_base.urdf ~/src/Firmware/Tools/sitl_gazebo/models/rotors_description/urdf/iris_base.xacro enable_mavlink_interface:=true enable_ground_truth:=false enable_wind:=false enable_logging:=false rotors_description_dir:=/home/mfaria/src/Firmware/Tools/sitl_gazebo/models/rotors_description`
        1. `sed -i 's/package:\/\/rotors_description/package:\/\/mavlink_sitl_gazebo\/models\/rotors_description/g' ~/src/Firmware/Tools/sitl_gazebo/models/rotors_description/urdf/iris_base.urdf`
        1. `export ROS_IP=192.168.6.77`
        1. `export ROS_MASTER_URI=http://192.168.6.231:11311`
    1. In Upboard add `192.168.6.77 garuda` to `/etc/hosts`


- real world [Hardware Setup - Try 2 CATEC](Paper%20-%20Exploration%20real%20world.ipynb#try2_catec)

<a id='run'></a>
#### 2.B Run

###### Hardware in the Loop (HitL)
          
1. **Laptop** Setup basics for px4 simulation
    1. `cd ~/src/Firmware`
    1. `source /opt/ros/kinetic/setup.bash`
    1. `source ~/Flying_Octomap_code/devel/setup.bash`
    1. `source Tools/setup_gazebo.bash ~/src/Firmware ~/src/Firmware/build/posix_sitl_default`
    1. `export ROS_PACKAGE_PATH=$ROS_PACKAGE_PATH:~/src/Firmware`
    1. `export ROS_PACKAGE_PATH=$ROS_PACKAGE_PATH:~/src/Firmware/Tools/sitl_gazebo`
    1. `cp ~/Flying_Octomap_code/src/architecture/Tools/iris.xacro ~/src/Firmware/Tools/sitl_gazebo/models/rotors_description/urdf/`
    1. `cp ~/Flying_Octomap_code/src/architecture/Tools/iris_base.xacro ~/src/Firmware/Tools/sitl_gazebo/models/rotors_description/urdf/`
    1. `python ~/src/Firmware/Tools/sitl_gazebo/scripts/xacro.py -o ~/src/Firmware/Tools/sitl_gazebo/models/rotors_description/urdf/iris_base.urdf ~/src/Firmware/Tools/sitl_gazebo/models/rotors_description/urdf/iris_base.xacro enable_mavlink_interface:=true enable_ground_truth:=false enable_wind:=false enable_logging:=false rotors_description_dir:=/home/garuda/src/Firmware/Tools/sitl_gazebo/models/rotors_description`
    1. `sed -i 's/package:\/\/rotors_description/package:\/\/mavlink_sitl_gazebo\/models\/rotors_description/g' ~/src/Firmware/Tools/sitl_gazebo/models/rotors_description/urdf/iris_base.urdf`
    1. `export ROS_IP=192.168.6.77`
    1. `export ROS_MASTER_URI=http://192.168.6.231:11311`

1. **UpBoard** Launch code
    1. `source ~/ros_ws/devel/setup.bash`
    1. `export ROS_IP=192.168.6.231`
    1. `roslaunch architecture hitl_upboard_architecture.launch`

1. **Laptop** Launch px4 simulation (2 terminals)
    1. `cd ~/src/Firmware`
    1. `roslaunch architecture posix_hitl_docker.launch`
    1. `roslaunch mavros px4.launch fcu_url:="udp://:14540@127.0.0.1:14557"`  

1. **Laptop** Launch rviz
    1. `cd ~/src/Firmware`
    1. `source /opt/ros/kinetic/setup.bash`
    1. `source ~/Flying_Octomap_code/devel/setup.bash`
    1. `source Tools/setup_gazebo.bash ~/src/Firmware ~/src/Firmware/build/posix_sitl_default`
    1. `export ROS_PACKAGE_PATH=$ROS_PACKAGE_PATH:~/src/Firmware`
    1. `export ROS_PACKAGE_PATH=$ROS_PACKAGE_PATH:~/src/Firmware/Tools/sitl_gazebo` 
    1. `export ROS_IP=192.168.6.77`
    1. `export ROS_MASTER_URI=http://192.168.6.231:11311`
    1. `rviz`
        
    


<a id='studyDesign'></a>
#### 2.C Study Design

Comparisions:
- It would be nice to compare against someone else's algorithm. MoveIt! seems to be the only 3D option of the shelf. See if this can be run from the bags. Also mention in future work that it will be compared - as it will...
- Regular grid & sparse?

20 runs from bag the see more statistically results.

Run around 3 in real life. Try to be with same flight.

Parameter sweep.

<a id='studyVariables'></a>
#### 2.D Study Variables

- Computation time
- RAM
- Path length free space
- Path lenght w/ obstacles: compare with manual measurement.
- Failure percentage
    - Naive max resolution neighbors
    - Different fractions of security margin

**Run plotly**
1. Installation
    1. `pip install plotly`
    1. `pip install cufflinks`
1. Run frontier extraction for box plots
    1. `cd ~/Flying_Octomap_code/_generate_plots`
    1. `python`
    1. `variables= {}`
    1. `execfile( "compare_box_graphs.py", variables )`